**Query decomposition**  is a strategy to improve question-answering using the power of LLM by re-writing and rephrasing the user input to improve retrieval process by breaking down a question into well-written sub-questions.

In [ ]:
pip install langchain langchain-community langchain-core langchain-openai langchain-text-splitters openai bs4 chromadb python-dotenv colorama tqdm tiktoken httplib2 langchainhub

utils.py

In [ ]:
from colorama import Fore


def format_qa_pair(question, answer):
    """Pairing and format Q and A"""
    
    formatted_string = ""
    formatted_string += f"{Fore.GREEN}Question: {question}{Fore.RESET}\n{Fore.WHITE}Answer: {answer}\n\n {Fore.RESET}"
    print("=====  QUESTION/ANSWER PAIRS: =====")
    print(formatted_string.strip())
    return formatted_string.strip()


def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

In [ ]:

import bs4
from dotenv import load_dotenv
from langchain import hub
from operator import itemgetter
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from colorama import Fore
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

# LLM
llm = ChatOpenAI()

In [ ]:
def index_documents(documents):
    # Index and load embeddings
    vectorstore = Chroma.from_documents(documents=documents, 
                                    embedding=OpenAIEmbeddings())

    # Create the vector store
    return vectorstore.as_retriever()

# 1. DECOMPOSITION

In [ ]:
template = """You are a helpful assistant trained to generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answered in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [ ]:
def  generate_sub_questions(query):
    generate_queries_decomposition = (
        prompt_decomposition 
        | llm 
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    ) 

    # Run
    sub_questions = generate_queries_decomposition.invoke({"question": query})
    questions_str = "\n".join(sub_questions)
    print(Fore.MAGENTA + "=====  SUBQUESTIONS: =====" + Fore.RESET)
    print(Fore.WHITE + questions_str + Fore.RESET + "\n") 
    return sub_questions

# 2. ANSWER SUBQUESTIONS RECURSIVELY 

In [ ]:
template = """Here is the question you need to answer:

\n --- \n {sub_question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {sub_question}
"""
prompt_qa = ChatPromptTemplate.from_template(template)

In [ ]:
def generate_qa_pairs(retriever, sub_questions):
    """ ask the LLM to generate a pair of question and answer based on the original user query """
    q_a_pairs = ""

    for sub_question in sub_questions:
        # chain
        generate_qa = (
            {"context": itemgetter("sub_question") | retriever, "sub_question": itemgetter("sub_question"), "q_a_pairs": itemgetter("q_a_pairs")}
            | prompt_qa 
            | llm 
            | StrOutputParser()
        )
        answer = generate_qa.invoke({"sub_question": sub_question, "q_a_pairs": q_a_pairs})
        q_a_pair = format_qa_pair(sub_question, answer)
        q_a_pairs = q_a_pairs + "\n --- \n" + q_a_pair 
    
    
        

# 3. ANSWER INDIVIDUALY

In [ ]:
from langchain import hub

In [ ]:
prompt_rag = hub.pull("rlm/rag-prompt")

In [ ]:
def retrieve_and_rag(retriever, sub_questions):
    rag_results = []
    for sub_question in sub_questions:
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        answer_chain = (
            prompt_rag
            | llm
            | StrOutputParser()
        )
        answer = answer_chain.invoke({"question": sub_question, "context": retrieved_docs})
        rag_results.append(answer)

    return rag_results, sub_questions

In [ ]:
pip install httplib2

In [ ]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer

http = httplib2.Http()

def get_links(url):
    status, response = http.request(url)
    links = []
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            links.append(f"https://www.paulgraham.com/{link.attrs['href']}")
    return links

In [ ]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import bs4

url = 'https://www.paulgraham.com/articles.html'

links = get_links(url)
loader = WebBaseLoader(
    web_paths=list(links),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("table")
        )
    ),
)

raw_text = loader.load()


# 4. SUMMARIZE AND ANSWER 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate


template = """Here is a set of Q+A pairs:
Paul Graham is known for his influential essays on startups, technology, programming, 
and life in general. He is a co-founder of Y Combinator, a prominent startup accelerator, 
and has written numerous essays that have inspired many entrepreneurs and technologists.
You assist users with general inquiries and {question} based on {context} /
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)


In [ ]:
def query(question):
    retriever = index_documents(raw_text)
    sub_questions = generate_sub_questions(query)
    generate_qa_pairs(retriever,  sub_questions)
    answers, questions = retrieve_and_rag(retriever, sub_questions)
    context = format_qa_pairs(questions, answers)

    final_rag_chain = (
        {"question": RunnablePassthrough(), "context": retriever}
        | chat_prompt_template
        | llm
        | StrOutputParser()
    )

    return final_rag_chain.invoke(question)

<span style="color:#8e44ad">Try it Out!</span>

In [ ]:
response = query("how do I write Python?")
print(f"{Fore.GREEN}{response}{Fore.RESET}")